<a href="https://colab.research.google.com/github/riow1983/pytorch_training/blob/master/CNN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
referece:
    https://blog.algorithmia.com/convolutional-neural-nets-in-pytorch/ 
"""

'\nreferece:\n    https://blog.algorithmia.com/convolutional-neural-nets-in-pytorch/ \n'

In [0]:
# Pytorch installation for Windows, Pip, Python3.7, CUDA=None
#!pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1-cp37-cp37m-win_amd64.whl
#!pip3 install torchvision

You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/fb/01/03fd7e503c16b3dc262483e5555ad40974ab5da8b9879e164b56c1f4ef6f/torchvision-0.2.2.post3-py2.py3-none-any.whl


You are using pip version 19.0.2, however version 19.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [0]:
#!python -V

Python 3.7.0


In [0]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms

In [0]:
import torch.nn.functional as F

In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./cifardata', train=True, download=True, transform=transform)
test_set = torchvision.datasets.CIFAR10(root='./cifardata', train=False, download=True, transform=transform)

0it [00:00, ?it/s]

170500096it [00:04, 36402165.16it/s]                               


Files already downloaded and verified


In [0]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [0]:
from torch.utils.data.sampler import SubsetRandomSampler

# Training
n_training_samples = 20000
train_sampler = SubsetRandomSampler(np.arange(n_training_samples, dtype=np.int64))

# Validation
n_val_samples = 5000
val_sampler = SubsetRandomSampler(np.arange(n_training_samples, n_training_samples + n_val_samples, dtype=np.int64))

# Test
n_test_samples = 5000
test_sampler = SubsetRandomSampler(np.arange(n_test_samples, dtype=np.int64))

In [0]:
from torch.autograd import Variable
import torch.nn.functional as F

class SimpleCNN(torch.nn.Module):
    
    # Our batch shape for input x is (3, 32, 32)
    
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        # Input channels = 3, output channels = 18
        self.conv1 = torch.nn.Conv2d(3, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        # 4608 input features, 64 output features (see sizing flow below)
        self.fc1 = torch.nn.Linear(18*16*16, 64)
        
        # 64 input features, 10 output features for our 10 defined classes
        self.fc2 = torch.nn.Linear(64, 10)
        
    def forward(self, x):
        
        # Computes the activation of the first convolution
        # Size changes from (3, 32, 32) to (18, 32, 32)
        x = F.relu(self.conv1(x))
        
        # Size changes from (18, 32, 32) to (18, 16, 16)
        x = self.pool(x)
        
        # Reshape data to input to the input layer of the neural net
        # Size changes from (18, 16, 16) to (1, 4608)
        x = x.view(-1, 18*16*16)
        
        # Computes the activation of the first fully connected layer
        # Size changes from (1, 4608) to (1, 64)
        x = F.relu(self.fc1(x))
        
        # Computes the second fully connected layer (activation applied later)
        # Size changes from (1, 64) to (1, 10)
        x = self.fc2(x)
        return(x)

In [0]:
def outputSize(in_size, kernel_size, stride, padding):
    output = int((in_size - kernel_size + 2*(padding)) / stride) + 1
    return(output)

# TO start, we'll define our data loaders using the samplers we created above

In [0]:
# Dataloader takes in a dataset and a sampler for loading (num_workers deals with system level memory)
def get_train_loader(batch_size):
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, sampler=train_sampler, num_workers=2)
    return(train_loader)

In [0]:
# Test and validation loaders have constant batch sizes, so we can define them directly
test_loader = torch.utils.data.DataLoader(test_set, batch_size=4, sampler=test_sampler, num_workers=2)
val_loader = torch.utils.data.DataLoader(train_set, batch_size=128, sampler=val_sampler, num_workers=2)

# Cross Entropy Loss (Log Loss)

In [0]:
import torch.optim as optim

def createLossAndOptimizer(net, learning_rate=0.001):
    
    # Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    # Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

# Train

In [0]:
import time

def trainNet(net, batch_size, n_epochs, learning_rate):
    
    # Print all of the hyperparameters of the training iteration:
    print("===== HYPERPARAMETERS =====")
    print("batch_size=", batch_size)
    print("epochs=", n_epochs)
    print("learning_rate=", learning_rate)
    print("=" * 30)
    
    # Get training data
    train_loader = get_train_loader(batch_size)
    n_batches = len(train_loader)
    
    # Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    # Time for printing
    training_start_time = time.time()
    
    # Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            # Get inputs
            inputs, labels = data
            
            # Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            # Set the parameters gradients to zero
            optimizer.zero_grad()
            
            # Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            # Print statistics
            running_loss += loss_size.data.item() #loss_size.data[0]
            total_train_loss += loss_size.data.item() #loss_size.data[0]
            
            # Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                    epoch+1, int(100*(i+1)/n_batches), running_loss / print_every, time.time() - start_time))
                # Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
                
        # At the end of the epoch, do a pass on the validation set 
        total_val_loss = 0
        for inputs, labels in val_loader:
            
            # Wrap tensors in Variables
            inputs, labels = Variable(inputs), Variable(labels)
            
            # Forward pass
            val_outputs = net(inputs)
            val_loss_size = loss(val_outputs, labels)
            total_val_loss += val_loss_size.data.item() #val_loss_size.data[0]
            
        print("Validation loss = {:.2f}".format(total_val_loss / len(val_loader)))
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))            

# Train actually

In [16]:
CNN = SimpleCNN()
trainNet(CNN, batch_size=32, n_epochs=5, learning_rate=0.001)

===== HYPERPARAMETERS =====
batch_size= 32
epochs= 5
learning_rate= 0.001
Epoch 1, 10% 	 train_loss: 2.08 took: 1.66s
Epoch 1, 20% 	 train_loss: 1.87 took: 1.61s
Epoch 1, 30% 	 train_loss: 1.73 took: 1.64s
Epoch 1, 40% 	 train_loss: 1.63 took: 1.66s
Epoch 1, 50% 	 train_loss: 1.58 took: 1.63s
Epoch 1, 60% 	 train_loss: 1.56 took: 1.65s
Epoch 1, 70% 	 train_loss: 1.50 took: 1.65s
Epoch 1, 80% 	 train_loss: 1.44 took: 1.69s
Epoch 1, 90% 	 train_loss: 1.45 took: 1.67s
Validation loss = 1.34
Epoch 2, 10% 	 train_loss: 1.37 took: 1.97s
Epoch 2, 20% 	 train_loss: 1.31 took: 2.73s
Epoch 2, 30% 	 train_loss: 1.32 took: 2.81s
Epoch 2, 40% 	 train_loss: 1.30 took: 2.55s
Epoch 2, 50% 	 train_loss: 1.24 took: 2.36s
Epoch 2, 60% 	 train_loss: 1.33 took: 2.34s
Epoch 2, 70% 	 train_loss: 1.25 took: 2.33s
Epoch 2, 80% 	 train_loss: 1.27 took: 2.29s
Epoch 2, 90% 	 train_loss: 1.30 took: 2.32s
Validation loss = 1.25
Epoch 3, 10% 	 train_loss: 1.19 took: 2.42s
Epoch 3, 20% 	 train_loss: 1.16 took: 2.35s


# Save

In [0]:
torch.save(CNN.state_dict(), "simpleCNN.pt")

In [20]:
#!ls

cifardata  sample_data	simpleCNN.pt


# Load



In [22]:
model = SimpleCNN()
model.load_state_dict(torch.load("simpleCNN.pt"))
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=4608, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

# Test

In [0]:
for inputs, labels in test_loader:
    # Wrap tensors in Variables
    inputs, labels = Variable(inputs), Variable(labels)

    # Forward pass
    test_outputs = model(inputs)

In [36]:
test_outputs.shape

torch.Size([4, 10])

In [44]:
torch.max(test_outputs, 1)[1]

tensor([5, 5, 8, 2])

In [28]:
labels

tensor([3, 4, 8, 6])

In [53]:
torch.max(test_outputs, 1)[1] == labels

tensor([0, 0, 1, 0], dtype=torch.uint8)